# LUT Generator

The purpose of this script is to generate a Look-Up Table (LUT) for the winning percentage of a given team at a given date in the season.  That way, when we have a game on which we're trying to predict the result, we have access to each team's winning percentage at the time of the game.  The LUT is saved as `lut.csv` at the end of the script.

We start by importing standard libraries, and reading in the games dataset.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('Data/games.csv')
df.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2021-05-26,42000102,Final,1610612755,1610612764,2020,1610612755,120.0,0.557,0.684,...,26.0,45.0,1610612764,95.0,0.402,0.633,0.091,22.0,40.0,1
1,2021-05-26,42000132,Final,1610612752,1610612737,2020,1610612752,101.0,0.383,0.739,...,15.0,54.0,1610612737,92.0,0.369,0.818,0.273,17.0,41.0,1
2,2021-05-26,42000142,Final,1610612762,1610612763,2020,1610612762,141.0,0.544,0.774,...,28.0,42.0,1610612763,129.0,0.541,0.763,0.348,20.0,33.0,1
3,2021-05-25,42000112,Final,1610612751,1610612738,2020,1610612751,130.0,0.523,0.955,...,31.0,46.0,1610612738,108.0,0.424,0.783,0.353,23.0,43.0,1
4,2021-05-25,42000152,Final,1610612756,1610612747,2020,1610612756,102.0,0.465,0.933,...,21.0,31.0,1610612747,109.0,0.450,0.871,0.303,24.0,39.0,0


Let's add a date column and pull out the year-month-day.  We'll then filter down to the 2018-19 season.

In [3]:
# make a date column
df['date'] = df['GAME_DATE_EST'].astype('datetime64')

# pull out useful features
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# filter for the 2018-19 NBA season
df = df.loc[(df['year'] == 2018) & \
               (((df['month'] == 10) & (df['day'] >= 16)) | \
               (df['month'] > 10)) | \
                      
              (df['year'] == 2019) & \
               (((df['month'] == 4) & (df['day'] <= 10)) | \
                (df['month'] < 4)), :]

# sort the games by date
df = df.sort_values('date')

df.head()

,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,date,year,month,day
18989,2018-10-16,21800002,Final,1610612744,1610612760,2018,1610612744,108.0,0.442,0.944,...,0.363,0.649,0.270,21.0,45.0,1,2018-10-16,2018,10,16
18988,2018-10-16,21800001,Final,1610612738,1610612755,2018,1610612738,105.0,0.433,0.714,...,0.391,0.609,0.192,18.0,47.0,1,2018-10-16,2018,10,16
18986,2018-10-17,21800012,Final,1610612746,1610612743,2018,1610612746,98.0,0.398,0.833,...,0.379,0.786,0.333,20.0,56.0,0,2018-10-17,2018,10,17
18977,2018-10-17,21800003,Final,1610612766,1610612749,2018,1610612766,112.0,0.446,0.636,...,0.494,0.750,0.412,26.0,57.0,0,2018-10-17,2018,10,17
18978,2018-10-17,21800004,Final,1610612765,1610612751,2018,1610612765,103.0,0.424,0.864,...,0.488,0.682,0.185,28.0,39.0,1,2018-10-17,2018,10,17


We will need to know which `team_id` corresponds to which team, and so we'll read in the `teams` dataset to generate a dictionary.

In [4]:
# read in the teams dataset
teams = pd.read_csv('Data/teams.csv')

# pull out the id and abbreviation columns, and convert to a dictionary
teams = teams[['TEAM_ID', 'ABBREVIATION']]
teams = teams.set_index('TEAM_ID')
teams = teams.to_dict()['ABBREVIATION']

teams

{1610612737: 'ATL',
 1610612738: 'BOS',
 1610612740: 'NOP',
 1610612741: 'CHI',
 1610612742: 'DAL',
 1610612743: 'DEN',
 1610612745: 'HOU',
 1610612746: 'LAC',
 1610612747: 'LAL',
 1610612748: 'MIA',
 1610612749: 'MIL',
 1610612750: 'MIN',
 1610612751: 'BKN',
 1610612752: 'NYK',
 1610612753: 'ORL',
 1610612754: 'IND',
 1610612755: 'PHI',
 1610612756: 'PHX',
 1610612757: 'POR',
 1610612758: 'SAC',
 1610612759: 'SAS',
 1610612760: 'OKC',
 1610612761: 'TOR',
 1610612762: 'UTA',
 1610612763: 'MEM',
 1610612764: 'WAS',
 1610612765: 'DET',
 1610612766: 'CHA',
 1610612739: 'CLE',
 1610612744: 'GSW'}

Now, we want a dictionary of DataFrames -- one for each team.  Each DataFrame will correspond to the collection of games that the team played that season, each row corresponding to a game.

In [5]:
# initialize the empty dictionary
data_dict = {}

# cycle through the different team IDs
for team_id in teams:
    
    # select out the games in which this team played
    data_dict[team_id] = df.loc[(df['HOME_TEAM_ID'] == team_id) | (df['VISITOR_TEAM_ID'] == team_id), :]
    
    # sort the games by date
    data_dict[team_id] = data_dict[team_id].sort_values('date').reset_index().drop('index', axis=1)

    # get rid of some columns we don't need here
    data_dict[team_id] = data_dict[team_id][['date', 'HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'PTS_home', 
                                             'PTS_away', 'HOME_TEAM_WINS']]
    
    # re-name the columns to easier names
    data_dict[team_id].rename(columns={'HOME_TEAM_ID' : 'home_id', 'VISITOR_TEAM_ID' : 'away_id', 'PTS_home' : 'home_pts', 
                                       'PTS_away' : 'away_pts', 'HOME_TEAM_WINS' : 'is_home_win'}, inplace=True)
    
    # add a column which tells us the team name for this dataframe
    data_dict[team_id]['team_name'] = teams[team_id]
    
    # re-order the columns to make it look nicer
    #data_dict[team_id] = data_dict[team_id][['team_name', 'date', 'home_id', 'away_id', 
    #                                         'home_pts', 'away_pts', 'is_home_win']]
    
    # get the winning percentage after each game 
    data_dict[team_id]['home_name'] = data_dict[team_id]['home_id'].map(teams)
    data_dict[team_id]['is_home'] = np.where(data_dict[team_id]['team_name'] == data_dict[team_id]['home_name'], 1, 0)
    data_dict[team_id]['is_win'] = np.where(data_dict[team_id]['is_home_win'] == data_dict[team_id]['is_home'], 1, 0)
    data_dict[team_id]['num_wins'] = data_dict[team_id]['is_win'].cumsum()
    data_dict[team_id]['num_games'] = data_dict[team_id].index + 1
    data_dict[team_id]['win_pct'] = round(data_dict[team_id]['num_wins'] / data_dict[team_id]['num_games'], 3)
    
    # select out the date and win_pct
    data_dict[team_id] = data_dict[team_id].loc[:, ['date', 'win_pct']]

Let's have a look at the first DataFrame:

In [6]:
test = data_dict[1610612737]
test.head()

,date,win_pct
0,2018-10-17,0.000
1,2018-10-19,0.000
2,2018-10-21,0.333
3,2018-10-24,0.500
4,2018-10-27,0.400


It looks like the DataFrame came out as expected (we can actually fact-check this by googling the results in this season, which I did separately and is not shown).  We'll now combine this with all of the days in the 2018-19 NBA season, to get the final Look-Up Table.

In [8]:
# initialize the LUT
lut = pd.DataFrame(pd.date_range(start='10/16/2018', end='4/10/2019'), columns=['date'])

# loop through each team in the dictionary of dataframes
for team_id in data_dict:
    
    team = teams[team_id]
    
    lut = pd.merge(lut, data_dict[team_id], how='left', on='date')
    lut.rename(columns={'win_pct' : team}, inplace=True)
    lut.fillna(method='ffill', inplace=True)
    lut.fillna(0.0, inplace=True)
    
display(lut.head())

,date,ATL,BOS,NOP,CHI,DAL,DEN,HOU,LAC,LAL,...,SAS,OKC,TOR,UTA,MEM,WAS,DET,CHA,CLE,GSW
0,2018-10-16,0.0,1.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,1.0
1,2018-10-17,0.0,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000,0.0,1.0
2,2018-10-18,0.0,1.000,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000,0.0,1.0
3,2018-10-19,0.0,0.500,1.0,0.0,0.0,1.0,0.0,0.5,0.0,...,1.0,0.0,1.0,0.5,0.5,0.0,1.0,0.500,0.0,1.0
4,2018-10-20,0.0,0.667,1.0,0.0,0.5,1.0,0.5,0.5,0.0,...,0.5,0.0,1.0,0.5,0.5,0.0,1.0,0.667,0.0,1.0


In [9]:
lut.shape

(177, 31)

It looks like we've generated the LUT properly! Let's save it as a new dataset now.

In [10]:
lut.to_csv('Data/lut.csv', index=False)